# Load structure

In [907]:
import crystal_toolkit

In [908]:
from ase.io import read
from ase import Atoms
from ase.build import add_adsorbate
from ase.geometry import get_distances
import numpy as np
import random

from pymatgen.core import Structure, Lattice, Molecule
from pymatgen.core.operations import SymmOp
# from pymatgen.transformations.standard_transformations import RotateMoleculeTransformation

In [909]:
# # Load supercell (either from ASE or convert from pymatgen)
# slab = read("supercell.xyz")  # or supercell.vasp etc.
# mol = read("triamine.xyz")    # your molecule

# # Optionally center molecule (important before rotation)
# mol.center(vacuum=0.0)

In [910]:
supercell = Structure.from_file("quantumatk/POSCAR_sc_li_outer")

In [911]:
print(supercell)

Full Formula (Li27 Ni27 O54)
Reduced Formula: LiNiO2
abc   :   8.716037  17.570024  26.187221
angles:  90.000000  90.000000  90.000002
pbc   :       True       True       True
Sites (108)
  #  SP            a         b         c
---  ----  ---------  --------  --------
  0  Li    -0         0.830726  0.078756
  1  Li    -0         0.497393  0.078756
  2  Li    -0         0.16406   0.078756
  3  Li     0.166666  0.748696  0.157512
  4  Li     0.166666  0.415363  0.157512
  5  Li     0.166667  0.08203   0.157512
  6  Li    -0         0.666666  0.236269
  7  Li    -0         0.333333  0.236269
  8  Li     0         0         0.236269
  9  Li     0.333333  0.830726  0.078756
 10  Li     0.333333  0.497393  0.078756
 11  Li     0.333333  0.16406   0.078756
 12  Li     0.5       0.748696  0.157512
 13  Li     0.5       0.415363  0.157512
 14  Li     0.5       0.08203   0.157512
 15  Li     0.333333  0.666666  0.236269
 16  Li     0.333333  0.333333  0.236269
 17  Li     0.333333  0         0

In [912]:
sc = read("quantumatk/POSCAR_sc_li_outer")

In [913]:
print(sc)

Atoms(symbols='Li27Ni27O54', pbc=True, cell=[[8.716036849615, 0.0, 0.0], [-4.66153e-07, 17.570024033931, 0.0], [0.0, 0.0, 26.187220967135]])


In [914]:
molecule = Molecule.from_file("fg/diurethane.xyz")
molecule

# ## alternative
# from pymatgen.io.ase import AseAtomsAdaptor

# atoms = read("fg/triamine.xyz")
# molecule_ase = AseAtomsAdaptor.get_molecule(atoms)

In [915]:
# molecule.center(vacuum=0.0)

In [916]:
molecule_centered = molecule.get_centered_molecule()
# print(molecule_centered)

In [917]:
# molecule_centered_aligned = molecule_centered.align_sites([0], [1]) # Note: doesn't exist
# print(molecule_centered_aligned)

In [918]:
mol = read("fg/triamine.xyz")

In [919]:
mol

Atoms(symbols='CH3CH2CH2NHNHNHCH3', pbc=False)

In [920]:
print(mol.center(vacuum=0.0))

None


In [921]:
def align_principal_axes(atoms: Atoms):
    pos = atoms.get_positions()
    pos -= pos.mean(axis=0)  # Center molecule

    # Compute inertia tensor
    inertia = np.dot(pos.T, pos)
    eigvals, eigvecs = np.linalg.eigh(inertia)

    # Align molecule along principal axes
    new_pos = np.dot(pos, eigvecs)
    atoms.set_positions(new_pos)
    return atoms

In [922]:
sc.cell

Cell([[8.716036849615, 0.0, 0.0], [-4.66153e-07, 17.570024033931, 0.0], [0.0, 0.0, 26.187220967135]])

In [923]:
# sc.cell.lenghts()     # Note: 'Cell' object has no attribute 'lenghts'

In [924]:
# # from ase.geometry import principal_axes

# # mol.center(vacuum=0.0)

# # Get principal axes of the molecule
# mol = align_principal_axes(mol)

# # Get lengths of supercell
# cell_lengths = sc.cell.lengths()
# longest_sc_axis = np.argmax(cell_lengths)

# # Rotate molecule so its longest principal axis aligns with sc's longest side
# from ase.geometry import rotate

# # Rotate mol's longest axis (axes[0]) to match e.g., [1, 0, 0] or [0, 1, 0]
# target_axes = [np.array([1, 0, 0]), np.array([0, 1, 0]), np.array([0, 0, 1])]
# rotate(mol, a1=axes[0], a2=target_axes[longest_sc_axis], rotate_cell=False)


In [925]:
# # Compute slab height
# z_max_slab = max(slab.positions[:, 2])

# # Shift molecule above the slab
# mol.translate([0, 0, z_max_slab + 3.0])  # 3 Angstrom above slab


In [926]:
# from ase import Atoms

# combined = slab + mol
# combined.wrap()  # optional: keep atoms inside cell

# # Save or view
# combined.write("combined_structure.xyz")


In [927]:
mol_xyz = align_principal_axes(mol)

In [928]:
mol_xyz

Atoms(symbols='CH3CH2CH2NHNHNHCH3', pbc=False, cell=[8.261140000000001, 3.7105300000000003, 2.79964])

In [929]:
mol.get_positions()

array([[-4.84726624e-02,  7.44472486e-01,  3.20577174e+00],
       [ 3.79576027e-01,  1.71029018e+00,  3.49050684e+00],
       [-1.14006889e+00,  8.41644370e-01,  3.21815345e+00],
       [ 2.31031671e-01, -9.92724746e-05,  3.97526094e+00],
       [-1.38346984e-01, -8.76944960e-01,  1.33905778e+00],
       [ 2.01252884e-01, -1.75399103e+00,  1.92167905e+00],
       [-1.23921179e+00, -8.25110802e-01,  1.43936419e+00],
       [-2.75688909e-01, -1.53870919e-01, -1.02329904e+00],
       [-2.16371075e-01,  8.55142117e-01, -5.87027987e-01],
       [-1.34156368e+00, -3.68654502e-01, -1.23011765e+00],
       [ 3.84491195e-01,  4.06783941e-01,  1.83702808e+00],
       [ 1.40307236e+00,  4.23932559e-01,  1.72627111e+00],
       [ 3.30524750e-01, -1.06032497e+00, -3.69382783e-02],
       [ 2.93653215e-01, -2.03164591e+00, -3.55017980e-01],
       [ 4.52705518e-01, -3.27762279e-01, -2.29079859e+00],
       [ 1.45268690e+00, -1.69013798e-01, -2.12939489e+00],
       [-7.18856762e-02,  4.96590331e-01

In [930]:
mol_xyz == mol

True

In [931]:
def substitute_ni_atoms(structure, nmc_ratio, seed=None):
    nmc_structure = structure.copy()
    
    nmc_ratio = np.array(nmc_ratio)
    nmc_ratio = nmc_ratio/nmc_ratio.sum()
    
    # Use a NumPy RNG like QuantumATK does
    rng = np.random.RandomState(seed)

    # Find all Ni atoms
    ni_indices = [i for i, site in enumerate(nmc_structure) if site.specie.symbol == "Ni"]
    print("Original Ni indices:", ni_indices)
    
    n_total = len(ni_indices)
    
    # Compute target numbers for each element
    ni_frac, mn_frac, co_frac = nmc_ratio
    n_ni = round(n_total * ni_frac)
    n_mn = round(n_total * mn_frac)
    n_co = n_total - n_ni - n_mn  # Ensure total is preserved
    
    # Shuffle Ni indices
    rng.shuffle(ni_indices)
    print("Shuffled Ni indices:", ni_indices)
    
    # Replace Ni with Mn
    for i in ni_indices[n_ni:n_ni + n_mn]:
        nmc_structure.replace(i, "Mn")
    
    # Replace Ni with Co
    for i in ni_indices[n_ni + n_mn:n_ni + n_mn + n_co]:
        nmc_structure.replace(i, "Co")
    
    print(f"NMC ratio: {nmc_ratio}")
    print(f"n_ni: {n_ni}, n_mn: {n_mn}, n_co: {n_co}")

    nmc_structure = nmc_structure.get_sorted_structure()
    
    return nmc_structure


In [932]:
nmc_structure = substitute_ni_atoms(structure=supercell, nmc_ratio=[1,1,1], seed=None)

Original Ni indices: [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]
Shuffled Ni indices: [44, 29, 40, 49, 36, 42, 51, 33, 52, 39, 35, 50, 53, 41, 30, 31, 34, 32, 48, 37, 43, 47, 46, 45, 28, 27, 38]
NMC ratio: [0.33333333 0.33333333 0.33333333]
n_ni: 9, n_mn: 9, n_co: 9


In [933]:
def get_principal_axis(mol: Molecule):
    coords = mol.cart_coords - mol.center_of_mass
    cov = np.cov(coords.T)
    eigvals, eigvecs = np.linalg.eigh(cov)
    # Return the eigenvector with the largest eigenvalue (longest axis)
    return eigvecs[:, np.argmax(eigvals)]


# Step 1: Get longest axis of molecule

In [934]:
molecule_axis = get_principal_axis(molecule)    # [-0.99961261 -0.02777762  0.00174037]
print(molecule_axis)

[-0.99984225 -0.0041892  -0.01726076]


In [935]:
# # Step 2: Get molecule's longest axis (in Cartesian)
# molecule_coords = np.array(molecule.cart_coords)
# molecule_extent = np.ptp(molecule_coords, axis=0)
# molecule_axis = molecule_extent / np.linalg.norm(molecule_extent)
# print(molecule_axis)

In [936]:
# molecule_coords = np.array(molecule.cart_coords)
# molecule_extent = np.ptp(molecule_coords, axis=0)
# molecule_axis = molecule_extent / np.linalg.norm(molecule_extent)     # array([0.8715153 , 0.39144521, 0.29535017])
# print(molecule_axis)

# Step 2: Determine which lattice direction has most layers

In [937]:
# Get longest lattice vector of supercell

# lattice = supercell.lattice.matrix
# lattice_lengths = np.linalg.norm(lattice, axis=1)
lattice_lengths = supercell.lattice.abc
longest_idx = np.argmax(lattice_lengths)
longest_unitvec = np.array([1 if i == longest_idx else 0 for i in range(3)])
# cell_axis = lattice[longest_idx] / lattice_lengths[longest_idx]     # returns: array([0., 0., 1.]) meaning z-axis as the longest


In [938]:
# Get direction of thickest layer

coords = np.array([site.frac_coords for site in supercell])
spread = np.ptp(coords, axis=0)  # peak-to-peak: max - min along each axis
thickest_idx = np.argmax(spread)
lattice_vec = supercell.lattice.matrix[thickest_idx]

# Get supercell direction (unit vector)
supercell_axis = lattice_vec / np.linalg.norm(lattice_vec)      # returns: array([-2.65311532e-08,  1.00000000e+00,  0.00000000e+00])

In [939]:
# Assuming molecule_axis and supercell_axis are normalized vectors
rotation_axis = np.cross(molecule_axis, supercell_axis)      # array([-0.02777762,  0.99961261,  0.        ])

# Step 3: Rotate molecule to align molecule_axis with supercell_axis

In [940]:
# Assuming molecule_axis and supercell_axis are normalized vectors
rotation_axis = np.cross(molecule_axis, supercell_axis)      # array([-0.02777762,  0.99961261,  0.        ])

In [941]:
# Skip rotation if vectors are already aligned
# if not np.isclose(rotation_angle_deg, 0):
if np.linalg.norm(rotation_axis) > 1e-3:
    rotation_axis /= np.linalg.norm(rotation_axis)
    rotation_angle_rad = np.arccos(np.clip(np.dot(molecule_axis, supercell_axis), -1.0, 1.0))    # 1.5690559577350829
    rotation_angle_deg = np.rad2deg(rotation_angle_rad)     # 89.90028419807753

    rotation = SymmOp.from_origin_axis_angle(origin=[0, 0, 0],
                                            axis=rotation_axis,
                                            angle=rotation_angle_deg,
                                            angle_in_radians=False)
    molecule_rotated = molecule.copy()
    molecule_rotated.apply_operation(rotation)
else:
    molecule_rotated = molecule.copy()

In [942]:
molecule_rotated

[2025-05-21 14:26:10,972] ERROR in app: Exception on /_dash-component-suites/dash/dash_table/async-highlight.js [GET]
Traceback (most recent call last):
  File "/home/azka/anaconda3/envs/full_automate/lib/python3.11/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/azka/anaconda3/envs/full_automate/lib/python3.11/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/azka/anaconda3/envs/full_automate/lib/python3.11/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/azka/anaconda3/envs/full_automate/lib/python3.11/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^

# Step 3a: flipped through a direction

In [ ]:
# Mirror over the y-axis (flip the sign of y-coordinates)
coords_flipped = molecule_rotated.cart_coords.copy()
# if molecule_name == "diurethane":
# elif molecule_name == "triamine":
coords_flipped[:, 0] *= -1  # Flip x-axis
coords_flipped[:, 1] *= -1  # Flip y-axis
# coords_flipped[:, 2] *= -1  # Flip z-axis
# elif molecule_name == "dicarbonate":
# unknown
# elif molecule_name == "triester": # approved
# elif molecule_name == "triether": # approved
# elif molecule_name == "triol": # approved

# Create a new Molecule with flipped coordinates
molecule_rotated_flipped = Molecule(species=molecule_rotated.species, coords=coords_flipped)
molecule_rotated_flipped

# Step 4: Translate molecule so it sits 3 Å above top of supercell

In [944]:
supercell_cart_coords = np.array([site.coords for site in supercell])
molecule_cart_coords = np.array(molecule_rotated_flipped.cart_coords)

# # --- Get topmost atom in slab along vacuum direction ---
# supercell_top = np.max(supercell_cart_coords[:, longest_idx])

# # Center of the supercell in y
# supercell_min = np.min(supercell_cart_coords[:, thickest_idx])
# supercell_max = np.max(supercell_cart_coords[:, thickest_idx])
# supercell_center = np.mean(supercell_cart_coords, axis=0)       # didnt really like this
# molecule_center = np.mean(molecule_cart_coords, axis=0)

In [945]:
print(supercell.cart_coords)

[[-3.44918910e-06  1.45958735e+01  2.06240286e+00]
 [-1.84113789e-06  8.73920320e+00  2.06240493e+00]
 [-2.33077957e-07  2.88253294e+00  2.06240699e+00]
 [ 1.45266948e+00  1.31546070e+01  4.12480985e+00]
 [ 1.45267108e+00  7.29793673e+00  4.12481192e+00]
 [ 1.45267269e+00  1.44126647e+00  4.12481398e+00]
 [-3.21611114e-06  1.17133405e+01  6.18721684e+00]
 [-1.60805993e-06  5.85667026e+00  6.18721890e+00]
 [ 0.00000000e+00  0.00000000e+00  6.18722097e+00]
 [ 2.90534217e+00  1.45958735e+01  2.06240286e+00]
 [ 2.90534378e+00  8.73920320e+00  2.06240493e+00]
 [ 2.90534538e+00  2.88253294e+00  2.06240699e+00]
 [ 4.35801509e+00  1.31546070e+01  4.12480985e+00]
 [ 4.35801670e+00  7.29793673e+00  4.12481192e+00]
 [ 4.35801831e+00  1.44126647e+00  4.12481398e+00]
 [ 2.90534240e+00  1.17133405e+01  6.18721684e+00]
 [ 2.90534401e+00  5.85667026e+00  6.18721890e+00]
 [ 2.90534562e+00  0.00000000e+00  6.18722097e+00]
 [ 5.81068778e+00  1.45958735e+01  2.06240286e+00]
 [ 5.81068939e+00  8.73920320e+

In [946]:
# --- Get molecule centroid and shift ---
molecule_com = molecule_rotated_flipped.center_of_mass
molecule_at_origin = molecule_rotated_flipped.translate_sites(range(len(molecule_rotated_flipped)), -molecule_com)

combined_structure = supercell.copy()
for site in molecule_at_origin.sites:
    combined_structure.append(site.specie, site.coords, coords_are_cartesian=True)

combined_structure = combined_structure.get_sorted_structure()

# Save result
combined_structure.to(filename="molecule_at_origin_center.cif")

"# generated using pymatgen\ndata_Li27Ni27H10C5(NO29)2\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   8.71603685\n_cell_length_b   17.57002403\n_cell_length_c   26.18722097\n_cell_angle_alpha   90.00000000\n_cell_angle_beta   90.00000000\n_cell_angle_gamma   90.00000152\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   Li27Ni27H10C5(NO29)2\n_chemical_formula_sum   'Li27 Ni27 H10 C5 N2 O58'\n_cell_volume   4010.33660195\n_cell_formula_units_Z   1\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  Li  Li0  1  -0.00000035  0.83072587  0.07875608  1\n  Li  Li1  1  -0.00000018  0.49739279  0.07875616  1\n  Li  Li2  1  -0.00000002  0.16405970  0.07875624  1\n  Li  Li3  1  0.16666632  0.74869602  0.15751232  1\n  Li  Li4  1  0.16666649  0.41536293  0.15751240  1\n 

In [947]:
# supercell center in 2D except the longest direction 
# longest_unitvec_inverse = 1 - longest_unitvec   # inversion of longest_unitvec
supercell_2d_center = supercell.lattice.matrix.sum(axis=0) / 2  # Question: how do they compute for y direc?
supercell_2d_center[longest_idx] = 0
print(supercell_2d_center)

[4.35801819 8.78501202 0.        ]


In [948]:
molecule_com

array([-0.02568162, -0.01571708, -0.00778793])

In [949]:
# Place molecule center in x and y center of supercell
molecule_translated_x_y = molecule_at_origin.copy()
translation = supercell_2d_center - molecule_com
molecule_translated_x_y.translate_sites(range(len(molecule_at_origin)), translation)

combined_structure = supercell.copy()
for site in molecule_translated_x_y.sites:
    combined_structure.append(site.specie, site.coords, coords_are_cartesian=True)

combined_structure = combined_structure.get_sorted_structure()

# Save result
combined_structure.to(filename="molecule_at_origin_center_translated_x_y.cif")

"# generated using pymatgen\ndata_Li27Ni27H10C5(NO29)2\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   8.71603685\n_cell_length_b   17.57002403\n_cell_length_c   26.18722097\n_cell_angle_alpha   90.00000000\n_cell_angle_beta   90.00000000\n_cell_angle_gamma   90.00000152\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   Li27Ni27H10C5(NO29)2\n_chemical_formula_sum   'Li27 Ni27 H10 C5 N2 O58'\n_cell_volume   4010.33660195\n_cell_formula_units_Z   1\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  Li  Li0  1  -0.00000035  0.83072587  0.07875608  1\n  Li  Li1  1  -0.00000018  0.49739279  0.07875616  1\n  Li  Li2  1  -0.00000002  0.16405970  0.07875624  1\n  Li  Li3  1  0.16666632  0.74869602  0.15751232  1\n  Li  Li4  1  0.16666649  0.41536293  0.15751240  1\n 

In [950]:
# Get z-coordinate of lowest atom in molecule
z_low_molecule = min(site.z for site in molecule_translated_x_y.sites)
print(z_low_molecule)

-1.36811259638381


In [951]:
# Then lift molecule along thickest_idx axis to be 3 Å above top

# Get z-coordinate of lowest atom in molecule
z_low_molecule = min(site.z for site in molecule_translated_x_y.sites)

# z_top_supercell = np.max([site.coords[longest_idx] for site in supercell.sites])
z_top_supercell = np.max(supercell_cart_coords[:, longest_idx])
# molecule_bottom = np.min([site[longest_idx] for site in molecule_rotated.cart_coords])
# z_shift = z_top_supercell - molecule_bottom + 3.0  # 3 Å vacuum
translation_z = np.zeros(3)
translation_z[longest_idx] = z_top_supercell - z_low_molecule + 3.0  # 3 Å vacuum

molecule_translated_x_y_z = molecule_translated_x_y.copy()
molecule_translated_x_y_z.translate_sites(range(len(molecule_translated_x_y)), translation_z)

combined_structure = supercell.copy()
for site in molecule_translated_x_y_z.sites:
    combined_structure.append(site.specie, site.coords, coords_are_cartesian=True)

combined_structure = combined_structure.get_sorted_structure()

# Save result
combined_structure.to(filename="molecule_at_origin_center_translated_x_y_z.cif")

"# generated using pymatgen\ndata_Li27Ni27H10C5(NO29)2\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   8.71603685\n_cell_length_b   17.57002403\n_cell_length_c   26.18722097\n_cell_angle_alpha   90.00000000\n_cell_angle_beta   90.00000000\n_cell_angle_gamma   90.00000152\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   Li27Ni27H10C5(NO29)2\n_chemical_formula_sum   'Li27 Ni27 H10 C5 N2 O58'\n_cell_volume   4010.33660195\n_cell_formula_units_Z   1\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  Li  Li0  1  -0.00000035  0.83072587  0.07875608  1\n  Li  Li1  1  -0.00000018  0.49739279  0.07875616  1\n  Li  Li2  1  -0.00000002  0.16405970  0.07875624  1\n  Li  Li3  1  0.16666632  0.74869602  0.15751232  1\n  Li  Li4  1  0.16666649  0.41536293  0.15751240  1\n 

In [952]:
# # Then lift molecule along longest axis to be 3 Å above top
# molecule_translated_x_y_z = molecule_translated_x_y.copy()
# offset = np.zeros(3)
# offset[longest_idx] = supercell_max - np.min(molecule_cart_coords[:, longest_idx]) + 3.0
# molecule_translated_x_y_z.translate_sites(range(len(molecule_translated_x_y)), offset)

# print("Note: doesn't work")

In [953]:
xx - yyyy

NameError: name 'xx' is not defined

# Combine

In [ ]:
molecule_structure = Structure(lattice=supercell.lattice, species=molecule_translated_x_y_z.species, coords=molecule_translated_x_y_z.cart_coords, coords_are_cartesian=True)

In [ ]:
molecule_structure

In [ ]:
combined_structure = supercell.copy()
for site in molecule_translated_x_y_z.sites:
    combined_structure.append(site.specie, site.coords, coords_are_cartesian=True)

combined_structure = combined_structure.get_sorted_structure()

# Save result
combined_structure.to(filename="combined_structure_v4.cif")
combined_structure.to(filename="POSCAR_combined_structure")

In [ ]:
# Combine structures
# combined_sites = list(supercell.sites) + list(molecule_structure.sites)
# final_structure = Structure.from_sites(combined_sites)

# # Save or visualize
# final_structure.to(filename="POSCAR_combined")

In [ ]:
final_structure

In [ ]:
combined